## 📊 Phase 5: Survey Analysis
Goal: Analyze customer survey responses to uncover satisfaction drivers, pain points, and actionable insights.

## **Phase 5 — Survey Analysis with Business Insights**

### **1. Data Cleaning & Preparation**

* Parse `Survey_Date` as `datetime`.
* Ensure `CSAT_Score` and `NPS_Score_Numeric` are numeric.
* Standardize `NPS_Category` values (Promoter, Passive, Detractor).
* Handle missing values (feedback text, scores).

---

### **2. Core KPIs**

* **Average CSAT Score** overall + trend over time.
* **NPS Score**:

  $$
  NPS = \%Promoters - \%Detractors
  $$

  * Trend chart of NPS over time.
* Distribution histograms for CSAT and NPS.

---

### **3. Segmentation (if extra columns exist)**

* CSAT & NPS by **subscription tier**.
* CSAT & NPS by **region**.
* Highlight top-performing & underperforming segments.

---

### **4. Text & Sentiment Analysis**

* Clean open-ended feedback (remove stopwords, lowercase).
* **Sentiment scoring** with polarity analysis.
* **Theme extraction** (top recurring phrases).
* **Visuals**:

  * Word cloud (most common feedback words).
  * Horizontal bar chart: top positive & negative themes.
  * Sentiment pie chart (positive/neutral/negative share).

---

### **5. Modern Visual Design**

* Consistent color theme (blue/orange/grey for professionalism).
* Large, readable axis labels and captions.
* Minimal clutter — focus on data story.

---

### **6. Business Interpretation Section**

* **Executive Summary**: One page with “key findings” in bullet form.
* **Insights**:

  * Main drivers of low NPS.
  * Customer strengths to maintain.
* **Recommendations**:

  * Action items to improve satisfaction.
  * Estimated business impact (qualitative).

---

### **7. Output**

* **`Survey_Insights_Report.pdf`** in `/output` folder:

  * Front page → Title, date, and high-level KPI summary.
  * Section 1 → NPS & CSAT trends.
  * Section 2 → Segmentation insights.
  * Section 3 → Sentiment & text analysis.
  * Section 4 → Recommendations.


In [2]:
"""
Phase 5 - Enhanced Survey Analysis with Modern Diagrams
Outputs:
  - output/Survey_Insights_Report.pdf
  - output/*.png charts
  - output/survey_top_words.csv
  - output/survey_top_bigrams.csv

Dependencies:
  pip install pandas numpy matplotlib seaborn vaderSentiment fpdf scikit-learn wordcloud
"""

import os
import re
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from fpdf import FPDF
from sklearn.feature_extraction.text import CountVectorizer

# -------- Settings --------
INPUT_CSV = "sports_survey_responses.csv"
OUTPUT_DIR = "output"
REPORT_PDF = os.path.join(OUTPUT_DIR, "Survey_Insights_Report.pdf")
TOP_K = 25
WORDCLOUD_AVAILABLE = True
try:
    from wordcloud import WordCloud
except Exception:
    WORDCLOUD_AVAILABLE = False

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Color palette for plots (professional)
PALETTE = ["#2b74d6", "#f28e2b", "#7f7f7f", "#59a14f"]

sns.set(style="whitegrid", context="talk", rc={"figure.dpi": 120})

# -------- Helpers --------
STOPWORDS = set([
    "the","and","a","an","is","it","to","for","of","i","we","you","this","that",
    "in","on","with","was","are","be","have","has","not","but","they","their",
    "my","me","our","as","at","from","by","so","if","or","can","all","too","no",
    "yes","very","just","like","get","got","also","will"
])

def safe_read_csv(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Input file not found: {path}")
    return pd.read_csv(path)

def clean_text(s):
    if pd.isna(s):
        return ""
    txt = str(s).lower()
    txt = re.sub(r"[^a-z0-9\s]", " ", txt)
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

def top_n_words(texts, n=20):
    words = []
    for t in texts:
        for w in t.split():
            if w in STOPWORDS or len(w) <= 2:
                continue
            words.append(w)
    return Counter(words).most_common(n)

def top_ngrams(texts, ngram_range=(2,2), n=20):
    vec = CountVectorizer(stop_words=list(STOPWORDS), ngram_range=ngram_range, min_df=1)
    X = vec.fit_transform(texts)
    sums = np.array(X.sum(axis=0)).flatten()
    terms = vec.get_feature_names_out()
    pairs = sorted(list(zip(terms, sums)), key=lambda x: x[1], reverse=True)
    return pairs[:n]

# -------- Load & Clean --------
df = safe_read_csv(INPUT_CSV)
data = df.copy()

# Normalize column names (helpful if CSV uses slightly different names)
col_map = {}
for c in data.columns:
    lc = c.strip().lower()
    if lc in ("customerid", "customer_id", "id"):
        col_map[c] = "CustomerID"
    elif lc in ("csat", "csat_score", "satisfaction"):
        col_map[c] = "CSAT_Score"
    elif lc in ("nps_category", "nps_cat"):
        col_map[c] = "NPS_Category"
    elif lc in ("nps_score", "nps_score_numeric", "nps"):
        col_map[c] = "NPS_Score_Numeric"
    elif lc in ("open_ended_feedback", "feedback", "comments", "open_feedback"):
        col_map[c] = "Open_Ended_Feedback"
    elif lc in ("survey_date", "date", "response_date"):
        col_map[c] = "Survey_Date"

if col_map:
    data = data.rename(columns=col_map)

required = ["CustomerID", "CSAT_Score", "NPS_Category", "NPS_Score_Numeric", "Open_Ended_Feedback", "Survey_Date"]
missing = [c for c in required if c not in data.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}. Found columns: {data.columns.tolist()}")

# Parse dates (try dayfirst then fallback)
data["Survey_Date"] = pd.to_datetime(data["Survey_Date"], dayfirst=True, errors="coerce")
if data["Survey_Date"].isna().any():
    data["Survey_Date"] = pd.to_datetime(data["Survey_Date"], dayfirst=False, errors="coerce")
if data["Survey_Date"].isna().any():
    bad = data[data["Survey_Date"].isna()]
    print(f"Warning: dropping {len(bad)} rows with unparsable Survey_Date")
    data = data.dropna(subset=["Survey_Date"]).reset_index(drop=True)

# Numeric enforcement
data["CSAT_Score"] = pd.to_numeric(data["CSAT_Score"], errors="coerce")
data["NPS_Score_Numeric"] = pd.to_numeric(data["NPS_Score_Numeric"], errors="coerce")

# Standardize NPS category: Promoter / Passive / Detractor
def normalize_nps(cat, num):
    if pd.isna(cat) and not pd.isna(num):
        try:
            s = int(num)
            if s >= 9:
                return "Promoter"
            elif s >= 7:
                return "Passive"
            else:
                return "Detractor"
        except:
            return "Unknown"
    if pd.isna(cat):
        return "Unknown"
    cc = str(cat).lower()
    if "promot" in cc:
        return "Promoter"
    if "pass" in cc:
        return "Passive"
    if "detr" in cc:
        return "Detractor"
    return normalize_nps(None, num)

data["NPS_Category"] = data.apply(lambda r: normalize_nps(r.get("NPS_Category"), r.get("NPS_Score_Numeric")), axis=1)

# Prepare monthly buckets for trend charts
data["month"] = data["Survey_Date"].dt.to_period("M").dt.to_timestamp()

# -------- Core Metrics --------
total_responses = len(data)
csat_avg = data["CSAT_Score"].dropna().mean()
csat_median = data["CSAT_Score"].dropna().median()

n_promoters = (data["NPS_Category"] == "Promoter").sum()
n_detractors = (data["NPS_Category"] == "Detractor").sum()
n_passives = (data["NPS_Category"] == "Passive").sum()
n_known = n_promoters + n_detractors + n_passives

if n_known > 0:
    pct_promoters = n_promoters / n_known * 100
    pct_detractors = n_detractors / n_known * 100
    nps_score = pct_promoters - pct_detractors
else:
    pct_promoters = pct_detractors = nps_score = np.nan

# Trends
csat_by_month = data.groupby("month")["CSAT_Score"].mean()
def nps_pct(series):
    k = series.dropna()
    if len(k) == 0:
        return np.nan
    prom = (k=="Promoter").sum()/len(k)*100
    det = (k=="Detractor").sum()/len(k)*100
    return prom - det
nps_by_month = data.groupby("month")["NPS_Category"].apply(nps_pct)

# -------- Text & Sentiment --------
analyzer = SentimentIntensityAnalyzer()
data["feedback_clean"] = data["Open_Ended_Feedback"].fillna("").apply(clean_text)
data["sentiment_score"] = data["feedback_clean"].apply(lambda t: analyzer.polarity_scores(t)["compound"] if t else 0.0)
def sentiment_label(s):
    if s >= 0.05:
        return "positive"
    elif s <= -0.05:
        return "negative"
    else:
        return "neutral"
data["sentiment_label"] = data["sentiment_score"].apply(sentiment_label)

sentiment_counts = data["sentiment_label"].value_counts().reindex(["positive","neutral","negative"]).fillna(0)

# Top words and bigrams
top_words = top_n_words(data["feedback_clean"], n=TOP_K)
top_bigrams = top_ngrams(data["feedback_clean"], ngram_range=(2,2), n=TOP_K)

top_words_df = pd.DataFrame(top_words, columns=["word","count"])
top_bigrams_df = pd.DataFrame(top_bigrams, columns=["phrase","count"])

top_words_df.to_csv(os.path.join(OUTPUT_DIR, "survey_top_words.csv"), index=False)
top_bigrams_df.to_csv(os.path.join(OUTPUT_DIR, "survey_top_bigrams.csv"), index=False)

# -------- Visuals (modern) --------
# 1) CSAT distribution (violin + strip)
plt.figure(figsize=(8,5))
sns.violinplot(x=data["CSAT_Score"], color=PALETTE[0], inner=None)
sns.stripplot(x=data["CSAT_Score"], color="k", size=3, jitter=0.2, alpha=0.6)
plt.title("CSAT Score Distribution")
plt.xlabel("CSAT Score")
plt.tight_layout()
csat_dist_path = os.path.join(OUTPUT_DIR, "csat_distribution.png")
plt.savefig(csat_dist_path)
plt.close()

# 2) NPS composition (bar)
plt.figure(figsize=(6,5))
sns.barplot(x=["Promoter","Passive","Detractor"], y=[n_promoters, n_passives, n_detractors], palette=PALETTE)
plt.title("NPS Composition (counts)")
plt.ylabel("Count")
plt.tight_layout()
nps_comp_path = os.path.join(OUTPUT_DIR, "nps_composition.png")
plt.savefig(nps_comp_path)
plt.close()

# 3) NPS over time
plt.figure(figsize=(10,5))
sns.lineplot(x=nps_by_month.index, y=nps_by_month.values, marker="o", color=PALETTE[1])
plt.title("NPS (Pct Promoters - Pct Detractors) by Month")
plt.xlabel("Month")
plt.ylabel("NPS (%)")
plt.xticks(rotation=45)
plt.tight_layout()
nps_time_path = os.path.join(OUTPUT_DIR, "nps_by_month.png")
plt.savefig(nps_time_path)
plt.close()

# 4) CSAT over time
plt.figure(figsize=(10,5))
sns.lineplot(x=csat_by_month.index, y=csat_by_month.values, marker="o", color=PALETTE[0])
plt.title("Average CSAT by Month")
plt.xlabel("Month")
plt.ylabel("Average CSAT")
plt.xticks(rotation=45)
plt.tight_layout()
csat_time_path = os.path.join(OUTPUT_DIR, "csat_by_month.png")
plt.savefig(csat_time_path)
plt.close()

# 5) Sentiment breakdown (donut)
plt.figure(figsize=(6,6))
vals = sentiment_counts.values
labels = sentiment_counts.index
colors = [PALETTE[1], PALETTE[2], PALETTE[3] if len(PALETTE)>3 else "#999999"]
plt.pie(vals, labels=labels, autopct="%1.1f%%", startangle=140, colors=[PALETTE[1], "#bdbdbd", PALETTE[2]])
# draw center circle
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title("Feedback Sentiment")
plt.tight_layout()
sentiment_path = os.path.join(OUTPUT_DIR, "sentiment_breakdown.png")
plt.savefig(sentiment_path)
plt.close()

# 6) Top words bar (horizontal)
if not top_words_df.empty:
    plt.figure(figsize=(8,6))
    sns.barplot(x="count", y="word", data=top_words_df.head(20), palette=PALETTE)
    plt.title("Top Feedback Words")
    plt.xlabel("Count")
    plt.ylabel("")
    plt.tight_layout()
    top_words_path = os.path.join(OUTPUT_DIR, "top_words_bar.png")
    plt.savefig(top_words_path)
    plt.close()
else:
    top_words_path = None

# 7) Top bigrams bar (horizontal)
if not top_bigrams_df.empty:
    plt.figure(figsize=(8,6))
    sns.barplot(x="count", y="phrase", data=top_bigrams_df.head(20), palette=PALETTE)
    plt.title("Top Feedback Phrases (Bigrams)")
    plt.xlabel("Count")
    plt.ylabel("")
    plt.tight_layout()
    top_bigrams_path = os.path.join(OUTPUT_DIR, "top_bigrams_bar.png")
    plt.savefig(top_bigrams_path)
    plt.close()
else:
    top_bigrams_path = None

# 8) Word cloud if available, fallback to top words image
wordcloud_path = os.path.join(OUTPUT_DIR, "wordcloud.png")
if WORDCLOUD_AVAILABLE and len(" ".join(data["feedback_clean"].tolist()).strip())>0:
    wc = WordCloud(width=1200, height=600, background_color="white", stopwords=STOPWORDS).generate(" ".join(data["feedback_clean"].tolist()))
    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(wordcloud_path)
    plt.close()
else:
    # fallback create a simple figure with top words (already saved above)
    wordcloud_path = top_words_path

# -------- Build Business-Focused PDF --------
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=12)

# Cover page
pdf.add_page()
pdf.set_font("Arial", "B", 18)
pdf.cell(0, 10, "Survey Insights Report", ln=1, align="C")
pdf.ln(4)
pdf.set_font("Arial", "", 11)
pdf.multi_cell(0, 6, f"Data source: {INPUT_CSV}")
pdf.multi_cell(0, 6, f"Total responses analyzed: {total_responses}")
pdf.multi_cell(0, 6, f"Average CSAT: {csat_avg:.2f}" if not np.isnan(csat_avg) else "Average CSAT: N/A")
pdf.multi_cell(0, 6, f"NPS (Pct Promoters - Pct Detractors): {nps_score:.1f}" if not np.isnan(nps_score) else "NPS: N/A")
pdf.ln(4)

# Executive summary
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 6, "Executive Summary", ln=1)
pdf.set_font("Arial", "", 11)
exec_lines = [
    f"- Average CSAT is {csat_avg:.2f}. This is a quick indicator of customer satisfaction.",
    f"- Computed NPS is {nps_score:.1f}. Positive values indicate more promoters than detractors.",
    f"- Sentiment in open feedback: {int(sentiment_counts.get('positive',0))} positive, {int(sentiment_counts.get('neutral',0))} neutral, {int(sentiment_counts.get('negative',0))} negative.",
    "- Top feedback themes and phrases are provided below; use them to guide product fixes and messaging."
]
for line in exec_lines:
    pdf.multi_cell(0, 6, line)
pdf.ln(3)

# Add visuals: arrange key charts
def add_image_if_exists(pdf_obj, img_path, w=180):
    if img_path and os.path.exists(img_path):
        try:
            pdf_obj.image(img_path, w=w, x=15)
            pdf_obj.ln(6)
        except Exception as e:
            print("Warning adding image to PDF:", e)

pdf.set_font("Arial", "B", 12)
pdf.cell(0, 6, "Key Visuals", ln=1)
pdf.ln(2)
add_image_if_exists(pdf, csat_dist_path, w=180)
add_image_if_exists(pdf, nps_comp_path, w=180)
add_image_if_exists(pdf, nps_time_path, w=180)

pdf.add_page()
add_image_if_exists(pdf, csat_time_path, w=180)
add_image_if_exists(pdf, sentiment_path, w=120)
add_image_if_exists(pdf, wordcloud_path, w=180)

pdf.add_page()
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 6, "Top Words and Phrases", ln=1)
pdf.ln(2)
add_image_if_exists(pdf, top_words_path, w=180)
add_image_if_exists(pdf, top_bigrams_path, w=180)

# Findings & recommendations
pdf.add_page()
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 6, "Findings and Recommendations", ln=1)
pdf.ln(2)
pdf.set_font("Arial", "", 11)
findings = [
    f"1) Overall CSAT average is {csat_avg:.2f}. If the number is below target (e.g., <4), prioritize product fixes and support improvements.",
    f"2) NPS is {nps_score:.1f}. If negative or low positive, focus on detractor reasons to reduce churn risk.",
    "3) Sentiment analysis shows the balance of positive and negative feedback. Address frequent negative themes quickly.",
    "4) Top bigram phrases indicate specific areas customers mention often. Use these to create targeted experiments or fixes.",
    "5) Recommended next steps: prioritize fixes from detractor feedback, run A/B tests on messaging for passives, and collect follow-up surveys after remediation."
]
for f in findings:
    pdf.multi_cell(0, 6, f)

# Save PDF
pdf.output(REPORT_PDF)
print("Report saved to:", REPORT_PDF)
print("Top words CSV:", os.path.join(OUTPUT_DIR, "survey_top_words.csv"))
print("Top bigrams CSV:", os.path.join(OUTPUT_DIR, "survey_top_bigrams.csv"))


C:\Users\Nicho\AppData\Local\Temp\ipykernel_5700\2595505228.py:110: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data["Survey_Date"] = pd.to_datetime(data["Survey_Date"], dayfirst=True, errors="coerce")
C:\Users\Nicho\AppData\Local\Temp\ipykernel_5700\2595505228.py:218: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=["Promoter","Passive","Detractor"], y=[n_promoters, n_passives, n_detractors], palette=PALETTE)
C:\Users\Nicho\AppData\Local\Temp\ipykernel_5700\2595505228.py:218: UserWarning: The palette list has more values (4) than needed (3), which may not be intended.
  sns.barplot(x=["Promoter","Passive","Detractor"], y=[n_promoters, n_passives, n_detractors], palette=PALETTE)
C:\Users\Nicho\AppData\Local\Temp\ipykernel_5700\2595505

Report saved to: output\Survey_Insights_Report.pdf
Top words CSV: output\survey_top_words.csv
Top bigrams CSV: output\survey_top_bigrams.csv
